# **Processador Quântico - Quantum Programs**

Neste notebook nós iremos praticar como usar programas quânticos para rodar operações em sequência nos qubits. Nós iremos usar esses programas para rodar o circuito do teleporte nos qubits dentro do processador.

In [1]:
import netsquid as ns
from netsquid.components.qprocessor import QuantumProcessor, PhysicalInstruction
from netsquid.components.qprogram import QuantumProgram
import netsquid.components.instructions as instr
import pydynaa

<br> <br> Neste exemplo, nós trabalharemos com programas quânticos que podemos executar em processadores. <br> <br>

<span style="color:red">
Como você já deve ter descoberto neste ponto, tentar executar instruções enquanto as instruções anteriores não foram concluídas causa erros (ProcessorBusyError). Aqui está um exemplo usando o processador do exemplo 1.<span>

In [2]:
phys_instructions1 = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3), # Inicializando um qubit
    PhysicalInstruction(instr.INSTR_H, duration=1),    # Porta H
    PhysicalInstruction(instr.INSTR_X, duration=1),    # Porta X
]

qproc1 = QuantumProcessor(name="ExampleQProcl",num_positions=1, phys_instructions=phys_instructions1)

<br> <br> <span style="color:red">
Tentando executar uma instrução enquanto outra não foi finalizada.</span>

In [3]:
qproc1.execute_instruction(instr.INSTR_X, [0]) # Agendando aplicar a porta X
qproc1.execute_instruction(instr.INSTR_H, [0]) # Agendando aplicar a porta H
ns.sim_run() # Rodando o próximo evento agendado

ProcessorBusyError: 

<span style = "color: green"> Para prevenir esse erro, nós precisamos rodar cada evento em sequência.</span>

Antes de tentarmos isso, vamos reiniciar o processador e também redefinir a linha do tempo da simulação, que limpa todos os eventos ainda agendados e zera o tempo da simulação.

In [4]:
qproc1.reset() # Reseta o processador
ns.sim_reset() # Reseta a timeline de simulação
ns.sim_time() # Printa o tempo

0.0

In [5]:
qproc1.execute_instruction(instr.INSTR_INIT, [0]) # Necessário para inicializar o qubit novamente, 3 ns
ns.sim_run() # Roda o próximo evento agendado
qproc1.execute_instruction(instr.INSTR_X, [0]) # Agenda aplicar a porta X, 1 ns
ns.sim_run() # Roda o próximo evento agendado
qproc1.execute_instruction(instr.INSTR_H, [0]) # Agenda aplicar a porta H, 1 ns
ns.sim_run() # Roda o próximo evento agendado
ns.sim_time()

5.0

<br> <br> Em vez de chamar ns.sim_run() entre cada instrução, podemos usar programas quânticos.
Os programas quânticos podem aplicar instruções sequencialmente. Vamos usar um programa quântico para teletransportar qubits como fizemos no Notebook D_QuantumMemory_Example3.

Seguimos novamente o diagrama do circuito <br> <br>

<img src = teleporte.png> <br> <br>

Em detalhes as etapas são: <br> <br>
- **Passo 1:** inicializar três qubits
- **Passo 2:** Aplique as portas H e S ao primeiro qubit para configurá-lo no estado que queremos teletransportar $|0_y\rangle = \frac{1}{\sqrt 2}(|0\rangle + i|1\rangle)$
- **Passo 3:** entrelaçar o qubit 2 e o qubit 3 no estado Bell $|\Phi_+\rangle = \frac{1}{\sqrt 2}(|00\rangle+|11\rangle) $ aplicando primeiro o H e a porta CNOT
- **Passo 4:** execute um BSM no qubit 1 e qubit 2
- **Passo 5:** aplicar correções condicionais ao qubit 3

<br> <br> Em vez de executar todas essas instruções sequencialmente no processador, agora escrevemos um programa quântico e apenas executamos esse programa.

<br> <br>$$|\Phi_+\rangle = \frac{1}{\sqrt 2}(|00\rangle+|11\rangle) $$

<br> <br>Para fazer isso, criamos aqui uma subclasse para a classe QuantumPorgram. Este programa será executado em três qubits e teletransportará o estado do qubit 1 (na posição de memória 0)
para qubit 3 (na posição de memória 2)

$$$$
<span style = "color: orange"> class netsquid.components.qprogram.QuantumProgram(num_qubits=None, parallel=True, qubit_mapping=None)
               </span>

In [6]:
class TeleportProgram(QuantumProgram):
    default_num_qubits =3 # número de qubits
    
    def program(self):
        q0, q1,q2 = self.get_qubit_indices(3) # verifica os índices dos qubits
        
        self.apply(instr.INSTR_INIT, [q0,q1,q2]) # inicializa os três qubits
        
        # conjunto de estados do qubit 1 pra y0
        self.apply(instr.INSTR_H, [q0]) # Aplica H em q0
        self.apply(instr.INSTR_S, [q0]) # Aplica S em q0
        
        # Emaranhando o qubit 2 e o qubit 3
        self.apply(instr.INSTR_H, [q1]) # Aplica H em q1
        self.apply(instr.INSTR_CNOT, [q1,q2]) # Aplica CNOT em q1 e q2
        
        # Agora emaranhando o qubit 1 e 2, e medindo ambos (BSM)
        self.apply(instr.INSTR_CNOT, [q0,q1])
        self.apply(instr.INSTR_H, [q0]) 
        self.apply(instr.INSTR_MEASURE,q0, output_key="m1") #medindo q0 e armazenando o resultado
        self.apply(instr.INSTR_MEASURE,q1, output_key="m2") #medindo q1 e armazenando o resultado
        
        yield self.run() # roda as instruções agendada acima
        
        # Aplicando as correções a depender da medição
        # Se a medição do qubit 2 (q1) resultar em '1', aplicamos a porta X ao qubit 3 (q2)
        if self.output["m2"][0] ==1:
            self.apply(instr.INSTR_X, q2)
        # Se a medição do qubit 1 (q0) resultar em '1', aplicamos a porta Z ao qubit 3 (q2)    
        if self.output["m1"][0]==1:
            self.apply(instr.INSTR_Z, q2)
            
        yield self.run() # Executa as instruções agendadas.

<br> <br> Para teletransportar o estado do qubit 1, nós precisamos apenas executar esse programa. Nós podemos executar esse programa em qualquer processador, desde que suporte as instruções. <br> <br>

Primeiro, nós especificamos as instruções físicas necessárias: <br> <br>

- Inicializando um qubit: 3 ns.
- Aplicando a porta S: 1 ns. Pode ser aplicada em paralelo, posição de memória 0
- Aplicando a porta H: 1 ns. Pode ser aplicada em paralelo, posição de memória 0,1
- Aplicando a porta X: 1 ns. Pode ser aplicada em paralelo, posição de memória 2
- Aplicando a porta Z: 1 ns. Pode ser aplicada em paralelo, posição de memória 2
- Aplicando a porta CNOT: 4 ns. Pode ser aplicada em paralelo, posição de memória (0 = controle, 1 = alvo) e (1 = controle, 2 = alvo)
- Medindo na base Z: 7 ns.

In [7]:
phys_instructions3 = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3),
    PhysicalInstruction(instr.INSTR_S, duration =1, parallel=True, topology=[0]),
    PhysicalInstruction(instr.INSTR_H, duration =1, parallel=True, topology=[0,1]),
    PhysicalInstruction(instr.INSTR_X, duration =1, parallel=True, topology=[2]),
    PhysicalInstruction(instr.INSTR_Z, duration =1, parallel=True, topology=[2]),
    PhysicalInstruction(instr.INSTR_CNOT, duration =1, parallel=True, topology=[(0,1),(1,2)]),
    PhysicalInstruction(instr.INSTR_MEASURE, duration =7, parallel=True, topology=[0,1])
]

<br> <br> A seguir, nós iniciaizaremos um processador quântico com uma memória quântica com três posições de memória, e utilizaremos as instruções físicas especificadas acima.

In [8]:
ns.sim_reset() # resetando a simulação

In [9]:
qproc3 = QuantumProcessor(name="ExampleQProc3", num_positions=3, phys_instructions = phys_instructions3)

<br> <br> Executamos agora o programa no processador que foi inicializado.

In [10]:
qproc3.execute_program(TeleportProgram()) # Agendando a execução do programa no processador
ns.sim_run() # Rodando o próximo evento agendado
ns.sim_time() # Checando o tempo atual em ns

14.0

Para avaliar quão bem o teleporte funcionou, olhamos a fidelidade.

In [12]:
qproc3_s3, = qproc3.peek(positions=[2])
print("Fidelity of teleported state: {}".format(ns.qubits.fidelity([qproc3_s3],ns.y0)))

Fidelity of teleported state: 0.9999999999999998


## **Sugestão de Prática**

Escreva um novo programa que ao invés de teleportar o estado ns.y0 $\rightarrow|0_y\rangle = \frac{1}{\sqrt 2}(|0\rangle + i|1\rangle)$ , teleporte o estado ns.h0 $\rightarrow|+\rangle = \frac{1}{\sqrt 2}(|0\rangle + |1\rangle)$ ou o estado ns.y1 $\rightarrow|1_y\rangle = \frac{1}{\sqrt 2}(|0\rangle - i|1\rangle)$. <br> <br>

Você pode também modificar o exemplo acima ou criar o seu próprio. Lembre-se de modificar as instruções físicas se necessário, e de trocar o estado alvo quando for calcular a fidelidade correspondente.